## The percentage that received treatment

In [1]:
PerReceived = 285/6247
PerReceived

0.04562189851128542

## The number of people have received treatment in 2020

In [2]:
NReceived = PerReceived*9100000
NReceived

415159.2764526973

## The average capacity of the facilities

In [3]:
Capacity = NReceived/226
Capacity

1836.9879489057403

## Import facility location data

In [5]:
import pandas as pd
FLdf = pd.read_csv("Facility_geographic_information_reduced.csv")
FL = FLdf[pd.notna(FLdf.Name)][["Name", "latitude","longitude"]]
FL['latitude'] = pd.to_numeric(FL['latitude'])

FL


,Name,latitude,longitude
0,Blick Center,41.094106,-81.537926
1,Child Guidance and Family Solutions,41.081061,-81.505427
2,Community Support Services,41.066365,-81.520584
3,OhioGuidestone,41.128681,-81.613532
4,Portage Path Behavioral Health Akron Clinic,41.076209,-81.519355
...,...,...,...
220,PsyCare,41.095712,-80.749544
221,Saint Elizabeth Hospital,41.114170,-80.658085
222,Allwell Behavioral Health Services,39.973110,-82.013014
223,Genesis Healthcare System,39.974802,-82.016605


## Import census tract data

In [5]:
import pandas as pd
CTdf = pd.read_csv("Census_Tracts_Data.csv")
CTdf = CTdf.query("State == 'Ohio'")
CT = CTdf[pd.notna(CTdf.GEOID)][["GEOID", "Tract_Lat","Tract_Long", "Tract_Population"]]

CT['GEOID']= CT['GEOID'].map(str)

CT


C:\Users\derek\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GEOID,Tract_Lat,Tract_Long,Tract_Population
50513,39001770100.0,38.956004,-83.355741,4493.0
50514,39001770200.0,38.990975,-83.551834,4998.0
50515,39001770300.0,38.840835,-83.583317,7133.0
50516,39001770400.0,38.771621,-83.544511,4149.0
50517,39001770500.0,38.750783,-83.364663,3567.0
...,...,...,...,...
53460,39175938100.0,40.960889,-83.394309,4230.0
53461,39175938200.0,40.810305,-83.409754,3846.0
53462,39175938300.0,40.850862,-83.287606,3624.0
53463,39175938400.0,40.805671,-83.281100,3251.0


## Combine the two dataframes together

In [6]:
FL['temp'] = 1
CT['temp'] = 1
newdf = pd.merge(FL, CT, on = ['temp'])
newdf = newdf.drop('temp',axis = 1)
newdf

,Name,latitude,longitude,GEOID,Tract_Lat,Tract_Long,Tract_Population
0,Blick Center,41.094106,-81.537926,39001770100.0,38.956004,-83.355741,4493.0
1,Blick Center,41.094106,-81.537926,39001770200.0,38.990975,-83.551834,4998.0
2,Blick Center,41.094106,-81.537926,39001770300.0,38.840835,-83.583317,7133.0
3,Blick Center,41.094106,-81.537926,39001770400.0,38.771621,-83.544511,4149.0
4,Blick Center,41.094106,-81.537926,39001770500.0,38.750783,-83.364663,3567.0
...,...,...,...,...,...,...,...
664195,National Youth Advocate Program Inc,39.941166,-82.006325,39175938100.0,40.960889,-83.394309,4230.0
664196,National Youth Advocate Program Inc,39.941166,-82.006325,39175938200.0,40.810305,-83.409754,3846.0
664197,National Youth Advocate Program Inc,39.941166,-82.006325,39175938300.0,40.850862,-83.287606,3624.0
664198,National Youth Advocate Program Inc,39.941166,-82.006325,39175938400.0,40.805671,-83.281100,3251.0


## Function that caculate the distance between each facilities and each census tracts

In [7]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [9]:
newdf['distance'] = [calculate_distance(newdf.iloc[i]['latitude'],newdf.iloc[i]['longitude'],newdf.iloc[i]['Tract_Lat'],newdf.iloc[i]['Tract_Long']) for i in range(newdf.shape[0])]
newdf

,Name,latitude,longitude,GEOID,Tract_Lat,Tract_Long,Tract_Population,distance
0,Blick Center,41.094106,-81.537926,39001770100.0,38.956004,-83.355741,4493.0,283.765134
1,Blick Center,41.094106,-81.537926,39001770200.0,38.990975,-83.551834,4998.0,290.037322
2,Blick Center,41.094106,-81.537926,39001770300.0,38.840835,-83.583317,7133.0,305.295544
3,Blick Center,41.094106,-81.537926,39001770400.0,38.771621,-83.544511,4149.0,309.855515
4,Blick Center,41.094106,-81.537926,39001770500.0,38.750783,-83.364663,3567.0,303.657510
...,...,...,...,...,...,...,...,...
664195,National Youth Advocate Program Inc,39.941166,-82.006325,39175938100.0,40.960889,-83.394309,4230.0,163.295111
664196,National Youth Advocate Program Inc,39.941166,-82.006325,39175938200.0,40.810305,-83.409754,3846.0,153.254851
664197,National Youth Advocate Program Inc,39.941166,-82.006325,39175938300.0,40.850862,-83.287606,3624.0,148.384846
664198,National Youth Advocate Program Inc,39.941166,-82.006325,39175938400.0,40.805671,-83.281100,3251.0,144.619109


## Clear out unnecessary columns

In [10]:
newdf = newdf.drop('latitude', axis = 1)
newdf = newdf.drop('longitude', axis = 1)
newdf = newdf.drop('Tract_Lat', axis = 1)
newdf = newdf.drop('Tract_Long', axis = 1)
newdf.to_csv("Distance.csv", index=False)

## Filter out the tracts that within the distance threshhold of facilities

In [6]:
newdf = pd.read_csv("Distance.csv")
df1 = newdf[(newdf['distance'] < 48.2803)]
df1

,Name,GEOID,Tract_Population,distance
368,Blick Center,3.903510e+10,2075.0,46.554776
369,Blick Center,3.903510e+10,4777.0,47.814144
370,Blick Center,3.903510e+10,2425.0,47.236864
371,Blick Center,3.903510e+10,2012.0,46.953560
372,Blick Center,3.903510e+10,2138.0,45.924221
...,...,...,...,...
663573,National Youth Advocate Program Inc,3.912797e+10,6549.0,28.512633
663574,National Youth Advocate Program Inc,3.912797e+10,5099.0,19.855966
663575,National Youth Advocate Program Inc,3.912797e+10,3989.0,31.667701
663576,National Youth Advocate Program Inc,3.912797e+10,6114.0,31.459974


## Obtain data with demand served column

In [8]:
df2 = df1
df2['demand_served'] = Capacity

Popsum = df2.groupby('Name').agg({'Tract_Population':['sum']})

df2 = df2.set_index('Name').join(Popsum, how='left')

df2['demand_avg'] = (df2['Tract_Population']*df2['demand_served'])/df2[('Tract_Population', 'sum')]

df2


C:\Users\derek\AppData\Local\Temp/ipykernel_16036/1116816935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['demand_served'] = Capacity
C:\Users\derek\anaconda3\lib\site-packages\pandas\core\frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,GEOID,Tract_Population,distance,demand_served,"(Tract_Population, sum)",demand_avg
Name,,,,,,
Access Ohio,3.900597e+10,3915.0,45.276696,1836.987949,539594.0,13.328183
Access Ohio,3.903397e+10,3428.0,42.913262,1836.987949,539594.0,11.670246
Access Ohio,3.903397e+10,4007.0,36.196106,1836.987949,539594.0,13.641387
Access Ohio,3.903397e+10,2876.0,30.514100,1836.987949,539594.0,9.791023
Access Ohio,3.903397e+10,3400.0,32.289170,1836.987949,539594.0,11.574923
...,...,...,...,...,...,...
Zepf Center/Woodruff,3.917302e+10,5743.0,31.865121,1836.987949,622259.0,16.954069
Zepf Center/Woodruff,3.917302e+10,3493.0,34.813228,1836.987949,622259.0,10.311782
Zepf Center/Woodruff,3.917302e+10,5749.0,34.039538,1836.987949,622259.0,16.971781


In [10]:
df2 = df2.rename({('Tract_Population', 'count'): 'Population_sum'}, axis=1)
Dserved = df2.groupby('GEOID').agg({'demand_avg':['sum']})
Dserved = Dserved.reset_index()
Dserved['GEOID'] = Dserved['GEOID'].apply(str)
Dserved

,GEOID,demand_avg
,,sum
0,39001770100.0,61.196519
1,39001770200.0,45.424549
2,39001770300.0,64.828593
3,39001770400.0,37.708374
4,39001770500.0,22.188895
...,...,...
2945,39175938100.0,22.200741
2946,39175938200.0,55.867215
2947,39175938300.0,54.516620


## Merge the new column with the original census tract data; data cleaning

In [11]:
CTdf['GEOID'] = CTdf['GEOID'].apply(str)
df3 = CTdf.set_index('GEOID').join(Dserved.set_index('GEOID'), how='left')
df3 = df3.rename({('demand_avg', 'sum'): 'demand_served'}, axis=1)
df3['demand_served'] = df3['demand_served'].fillna(0)
df3

C:\Users\derek\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,County Code,Census_Tract,County,State,Tract_Population,Tract_Lat,Tract_Long,County_Population,State_Population,SMI_Estimates,SMI_Tract,%Population,OP_clients,unmet,Seek_for_help,demand_served
GEOID,,,,,,,,,,,,,,,,
39001770100.0,39001.0,Census Tract 7701,Adams County,Ohio,4493.0,38.956004,-83.355741,27926.0,11609756.0,6.38%,286.6534,0.000387,91.853014,142.466740,234.319754,61.196519
39001770200.0,39001.0,Census Tract 7702,Adams County,Ohio,4998.0,38.990975,-83.551834,27926.0,11609756.0,6.38%,318.8724,0.000431,102.177023,158.479583,260.656606,45.424549
39001770300.0,39001.0,Census Tract 7703,Adams County,Ohio,7133.0,38.840835,-83.583317,27926.0,11609756.0,6.38%,455.0854,0.000614,145.824071,226.177444,372.001515,64.828593
39001770400.0,39001.0,Census Tract 7704,Adams County,Ohio,4149.0,38.771621,-83.544511,27926.0,11609756.0,6.38%,264.7062,0.000357,84.820422,131.558981,216.379404,37.708374
39001770500.0,39001.0,Census Tract 7705,Adams County,Ohio,3567.0,38.750783,-83.364663,27926.0,11609756.0,6.38%,227.5746,0.000307,72.922257,113.104576,186.026834,22.188895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39175938100.0,39175.0,Census Tract 9381,Wyandot County,Ohio,4230.0,40.960889,-83.394309,22190.0,11609756.0,6.52%,275.7960,0.000364,86.476352,137.070612,223.546964,22.200741
39175938200.0,39175.0,Census Tract 9382,Wyandot County,Ohio,3846.0,40.810305,-83.409754,22190.0,11609756.0,6.52%,250.7592,0.000331,78.626017,124.627322,203.253339,55.867215
39175938300.0,39175.0,Census Tract 9383,Wyandot County,Ohio,3624.0,40.850862,-83.287606,22190.0,11609756.0,6.52%,236.2848,0.000312,74.087542,117.433546,191.521087,54.516620


## Obtain the demand left of each tracts

In [12]:
df4 = df3
df4['demand_left'] = df3['SMI_Tract']-df3['demand_served']
df4 = df4.reset_index()
df4['GEOID'] = df4['GEOID'].astype('string')
df4

,GEOID,County Code,Census_Tract,County,State,Tract_Population,Tract_Lat,Tract_Long,County_Population,State_Population,SMI_Estimates,SMI_Tract,%Population,OP_clients,unmet,Seek_for_help,demand_served,demand_left
0,39001770100.0,39001.0,Census Tract 7701,Adams County,Ohio,4493.0,38.956004,-83.355741,27926.0,11609756.0,6.38%,286.6534,0.000387,91.853014,142.466740,234.319754,61.196519,225.456881
1,39001770200.0,39001.0,Census Tract 7702,Adams County,Ohio,4998.0,38.990975,-83.551834,27926.0,11609756.0,6.38%,318.8724,0.000431,102.177023,158.479583,260.656606,45.424549,273.447851
2,39001770300.0,39001.0,Census Tract 7703,Adams County,Ohio,7133.0,38.840835,-83.583317,27926.0,11609756.0,6.38%,455.0854,0.000614,145.824071,226.177444,372.001515,64.828593,390.256807
3,39001770400.0,39001.0,Census Tract 7704,Adams County,Ohio,4149.0,38.771621,-83.544511,27926.0,11609756.0,6.38%,264.7062,0.000357,84.820422,131.558981,216.379404,37.708374,226.997826
4,39001770500.0,39001.0,Census Tract 7705,Adams County,Ohio,3567.0,38.750783,-83.364663,27926.0,11609756.0,6.38%,227.5746,0.000307,72.922257,113.104576,186.026834,22.188895,205.385705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,39175938100.0,39175.0,Census Tract 9381,Wyandot County,Ohio,4230.0,40.960889,-83.394309,22190.0,11609756.0,6.52%,275.7960,0.000364,86.476352,137.070612,223.546964,22.200741,253.595259
2948,39175938200.0,39175.0,Census Tract 9382,Wyandot County,Ohio,3846.0,40.810305,-83.409754,22190.0,11609756.0,6.52%,250.7592,0.000331,78.626017,124.627322,203.253339,55.867215,194.891985
2949,39175938300.0,39175.0,Census Tract 9383,Wyandot County,Ohio,3624.0,40.850862,-83.287606,22190.0,11609756.0,6.52%,236.2848,0.000312,74.087542,117.433546,191.521087,54.516620,181.768180
2950,39175938400.0,39175.0,Census Tract 9384,Wyandot County,Ohio,3251.0,40.805671,-83.281100,22190.0,11609756.0,6.52%,211.9652,0.000280,66.462085,105.346704,171.808790,61.014764,150.950436


In [13]:
df4.to_csv("Dserved.csv", index=False)

In [14]:
import pandas as pd
CTdf = pd.read_csv("Census_Tracts_Data.csv")
CTdf = CTdf.query("State == 'Ohio'")
CT = CTdf[pd.notna(CTdf.GEOID)][["GEOID", "Tract_Lat","Tract_Long", "Tract_Population"]]

CT['GEOID']= CT['GEOID'].map(str)
CT2 = CT

C:\Users\derek\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
CT['temp'] = 1
CT2['temp'] = 1
newdf2 = pd.merge(CT, CT2, on = ['temp'])
newdf2 = newdf2.drop('temp',axis = 1)
newdf2

,GEOID_x,Tract_Lat_x,Tract_Long_x,Tract_Population_x,GEOID_y,Tract_Lat_y,Tract_Long_y,Tract_Population_y
0,39001770100.0,38.956004,-83.355741,4493.0,39001770100.0,38.956004,-83.355741,4493.0
1,39001770100.0,38.956004,-83.355741,4493.0,39001770200.0,38.990975,-83.551834,4998.0
2,39001770100.0,38.956004,-83.355741,4493.0,39001770300.0,38.840835,-83.583317,7133.0
3,39001770100.0,38.956004,-83.355741,4493.0,39001770400.0,38.771621,-83.544511,4149.0
4,39001770100.0,38.956004,-83.355741,4493.0,39001770500.0,38.750783,-83.364663,3567.0
...,...,...,...,...,...,...,...,...
8714299,39175938500.0,40.761518,-83.214253,3057.0,39175938100.0,40.960889,-83.394309,4230.0
8714300,39175938500.0,40.761518,-83.214253,3057.0,39175938200.0,40.810305,-83.409754,3846.0
8714301,39175938500.0,40.761518,-83.214253,3057.0,39175938300.0,40.850862,-83.287606,3624.0
8714302,39175938500.0,40.761518,-83.214253,3057.0,39175938400.0,40.805671,-83.281100,3251.0


In [16]:
newdf2['distance'] = [calculate_distance(newdf2.iloc[i]['Tract_Lat_x'],newdf2.iloc[i]['Tract_Long_x'],newdf2.iloc[i]['Tract_Lat_y'],newdf2.iloc[i]['Tract_Long_y']) for i in range(newdf2.shape[0])]
newdf2

,GEOID_x,Tract_Lat_x,Tract_Long_x,Tract_Population_x,GEOID_y,Tract_Lat_y,Tract_Long_y,Tract_Population_y,distance
0,39001770100.0,38.956004,-83.355741,4493.0,39001770100.0,38.956004,-83.355741,4493.0,0.000000
1,39001770100.0,38.956004,-83.355741,4493.0,39001770200.0,38.990975,-83.551834,4998.0,17.397402
2,39001770100.0,38.956004,-83.355741,4493.0,39001770300.0,38.840835,-83.583317,7133.0,23.499022
3,39001770100.0,38.956004,-83.355741,4493.0,39001770400.0,38.771621,-83.544511,4149.0,26.227910
4,39001770100.0,38.956004,-83.355741,4493.0,39001770500.0,38.750783,-83.364663,3567.0,22.839743
...,...,...,...,...,...,...,...,...,...
8714299,39175938500.0,40.761518,-83.214253,3057.0,39175938100.0,40.960889,-83.394309,4230.0,26.855135
8714300,39175938500.0,40.761518,-83.214253,3057.0,39175938200.0,40.810305,-83.409754,3846.0,17.335964
8714301,39175938500.0,40.761518,-83.214253,3057.0,39175938300.0,40.850862,-83.287606,3624.0,11.700352
8714302,39175938500.0,40.761518,-83.214253,3057.0,39175938400.0,40.805671,-83.281100,3251.0,7.470996


In [18]:
check = newdf2

In [26]:
newdf2['GEOID_x'] = newdf2['GEOID_x'].astype('string')
newdf2['GEOID_y'] = newdf2['GEOID_y'].astype('string')

In [21]:
newdf2 = newdf2.drop('Tract_Lat_x', axis = 1)
newdf2 = newdf2.drop('Tract_Long_x', axis = 1)
newdf2 = newdf2.drop('Tract_Lat_y', axis = 1)
newdf2 = newdf2.drop('Tract_Long_y', axis = 1)
newdf2 = newdf2.drop('Tract_Population_x', axis = 1)
newdf2 = newdf2.drop('Tract_Population_y', axis = 1)

In [23]:
newdf2.to_csv("TractDistance.csv", index=False)

In [24]:
import pandas as pd
newdf2 = pd.read_csv("TractDistance.csv")
newdf2

,GEOID_x,GEOID_y,distance
0,3.900177e+10,3.900177e+10,0.000000
1,3.900177e+10,3.900177e+10,17.397402
2,3.900177e+10,3.900177e+10,23.499022
3,3.900177e+10,3.900177e+10,26.227910
4,3.900177e+10,3.900177e+10,22.839743
...,...,...,...
8714299,3.917594e+10,3.917594e+10,26.855135
8714300,3.917594e+10,3.917594e+10,17.335964
8714301,3.917594e+10,3.917594e+10,11.700352
8714302,3.917594e+10,3.917594e+10,7.470996


In [13]:
newdf2.loc[newdf2['GEOID_x'] == '39165032201.0']['distance'].sum()

15480.444885398108

In [9]:
newdf2 = newdf2[newdf2['distance'] <= 48.2803] 
newdf2

,GEOID_x,GEOID_y,distance
0,39001770100.0,39001770100.0,0.000000
1,39001770200.0,39001770200.0,17.397402
2,39001770300.0,39001770300.0,23.499022
3,39001770400.0,39001770400.0,26.227910
4,39001770500.0,39001770500.0,22.839743
...,...,...,...
8714299,39175938100.0,39175938100.0,26.855135
8714300,39175938200.0,39175938200.0,17.335964
8714301,39175938300.0,39175938300.0,11.700352
8714302,39175938400.0,39175938400.0,7.470996


In [8]:
newdf2['distance'].sum()

26127580.494091

In [42]:
newdf3 = pd.read_csv("Dserved.csv")
newdf3

,GEOID,County Code,Census_Tract,County,State,Tract_Population,Tract_Lat,Tract_Long,County_Population,State_Population,SMI_Estimates,SMI_Tract,%Population,OP_clients,unmet,Seek_for_help,demand_served,demand_left
0,3.900177e+10,39001.0,Census Tract 7701,Adams County,Ohio,4493.0,38.956004,-83.355741,27926.0,11609756.0,6.38%,286.6534,0.000387,91.853014,142.466740,234.319754,61.196519,225.456881
1,3.900177e+10,39001.0,Census Tract 7702,Adams County,Ohio,4998.0,38.990975,-83.551834,27926.0,11609756.0,6.38%,318.8724,0.000431,102.177023,158.479583,260.656606,45.424549,273.447851
2,3.900177e+10,39001.0,Census Tract 7703,Adams County,Ohio,7133.0,38.840835,-83.583317,27926.0,11609756.0,6.38%,455.0854,0.000614,145.824071,226.177444,372.001515,64.828593,390.256807
3,3.900177e+10,39001.0,Census Tract 7704,Adams County,Ohio,4149.0,38.771621,-83.544511,27926.0,11609756.0,6.38%,264.7062,0.000357,84.820422,131.558981,216.379404,37.708374,226.997826
4,3.900177e+10,39001.0,Census Tract 7705,Adams County,Ohio,3567.0,38.750783,-83.364663,27926.0,11609756.0,6.38%,227.5746,0.000307,72.922257,113.104576,186.026834,22.188895,205.385705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,3.917594e+10,39175.0,Census Tract 9381,Wyandot County,Ohio,4230.0,40.960889,-83.394309,22190.0,11609756.0,6.52%,275.7960,0.000364,86.476352,137.070612,223.546964,22.200741,253.595259
2948,3.917594e+10,39175.0,Census Tract 9382,Wyandot County,Ohio,3846.0,40.810305,-83.409754,22190.0,11609756.0,6.52%,250.7592,0.000331,78.626017,124.627322,203.253339,55.867215,194.891985
2949,3.917594e+10,39175.0,Census Tract 9383,Wyandot County,Ohio,3624.0,40.850862,-83.287606,22190.0,11609756.0,6.52%,236.2848,0.000312,74.087542,117.433546,191.521087,54.516620,181.768180
2950,3.917594e+10,39175.0,Census Tract 9384,Wyandot County,Ohio,3251.0,40.805671,-83.281100,22190.0,11609756.0,6.52%,211.9652,0.000280,66.462085,105.346704,171.808790,61.014764,150.950436


In [27]:
newdf2

,GEOID_x,GEOID_y,distance
0,39001770100.0,39001770100.0,0.000000
1,39001770100.0,39001770200.0,17.397402
2,39001770100.0,39001770300.0,23.499022
3,39001770100.0,39001770400.0,26.227910
4,39001770100.0,39001770500.0,22.839743
...,...,...,...
8714299,39175938500.0,39175938100.0,26.855135
8714300,39175938500.0,39175938200.0,17.335964
8714301,39175938500.0,39175938300.0,11.700352
8714302,39175938500.0,39175938400.0,7.470996


In [51]:
newdf3['GEOID'] = newdf3['GEOID'].astype('string')
